In [11]:
#!/usr/bin/env python
"""
NH₃ deposition → Natura-2000 bogs (top-3 by area)
"""

import geopandas as gpd
import xarray as xr
import numpy as np
import regionmask
import pandas as pd

# ------------------------------------------------------------------ #
# 1.  Load shapefile and select largest three polygons               #
# ------------------------------------------------------------------ #
shp = "spain_natura_2000b_bogs/spain_natura_2000b_bogs.shp"
gdf = gpd.read_file(shp).to_crs("EPSG:4326")          # ensure lat/lon
gdf["area_km2"] = gdf.geometry.area * (111**2)        # rough km² in lat/lon
top3 = gdf.nlargest(3, "area_km2").reset_index(drop=True)

# Give each polygon an ID and a nicer name
top3["name"] = [f"{row['SITENAME'][:25].strip()}…" if 'SITENAME' in row else f"reserve_{i}"
                for i, row in top3.iterrows()]
top3["id"]   = range(1, 4)

# ------------------------------------------------------------------ #
# 2.  Build a regionmask on the NH₃ grid                             #
# ------------------------------------------------------------------ #
ds  = xr.open_dataset("NH3_yearly_dep_2019-2022.nc")  # yearly totals
# mask = regionmask.Regions_cls(name="n2000",
#                               numbers=top3["id"],
#                               names=top3["name"],
#                               outlines=top3.geometry.values)
# mask_2d = mask.mask(ds, lat_name="lat", lon_name="lon")   # dims: lat, lon

# OLD (raises AttributeError)
# mask = regionmask.Regions_cls(name="n2000", numbers=top3["id"],
#                               names=top3["name"], outlines=top3.geometry.values)
# mask_2d = mask.mask(ds, lat_name="lat", lon_name="lon")

# Build Regions from the GeoDataFrame (top3 already contains 'id' and 'name')
reg = regionmask.from_geopandas(top3, numbers="id", names="name", name="n2000")

# New API: just pass the Dataset; regionmask figures out lat/lon names itself
mask_2d = reg.mask(ds)          # dims: lat × lon  (2-D integer mask)

# ------------------------------------------------------------------ #
# 3.  Area of each grid cell (deg² → m²); needed for totals          #
# ------------------------------------------------------------------ #
# 0.1° grid -> ~12.3 km × 11.1 km at 40°N; use xESMF area if you have it
R = 6_371_000                                   # Earth radius (m)
dlat = np.deg2rad(np.diff(ds.lat)[0])
dlon = np.deg2rad(np.diff(ds.lon)[0])
lat_factor = np.cos(np.deg2rad(ds.lat))     # 1-D (lat)
cell_area = (R**2 * dlat * dlon) * lat_factor.broadcast_like(ds.NH3_dry_dep.isel(year=0))
# cell_area dims: ('lat','lon')

# ------------------------------------------------------------------ #
# 4.  Loop over years & reserves and sum kg NH3 per reserve          #
# ------------------------------------------------------------------ #
records = []
for yr in ds.year.values:
    for rid, name in zip(top3.id, top3.name):
        m = (mask_2d == rid)
        dry_kg = (ds.NH3_dry_dep.sel(year=yr)*cell_area).where(m).sum().item()
        wet_kg = (ds.NH3_wet_dep.sel(year=yr)*cell_area).where(m).sum().item()
        records.append(dict(year=int(yr), reserve=name,
                            dry_kg=dry_kg, wet_kg=wet_kg,
                            total_kg=dry_kg+wet_kg))

df = pd.DataFrame(records)
df.to_csv("NH3_dep_per_reserve.csv", index=False)

# ------------------------------------------------------------------ #
# 5.  Save per-reserve, per-year arrays to NetCDF (optional)         #
# ------------------------------------------------------------------ #
da_dry = xr.DataArray(
    np.full((len(ds.year), 3), np.nan),
    coords=dict(year=ds.year, reserve=top3.name),
    name="dry_dep_kg")
da_wet = xr.DataArray(
    np.full((len(ds.year), 3), np.nan),
    coords=dict(year=ds.year, reserve=top3.name),
    name="wet_dep_kg")

for rec in records:
    y = rec["year"]; r = rec["reserve"]
    da_dry.loc[dict(year=y, reserve=r)] = rec["dry_kg"]
    da_wet.loc[dict(year=y, reserve=r)] = rec["wet_kg"]

xr.Dataset({"dry_dep_kg": da_dry, "wet_dep_kg": da_wet}) \
  .to_netcdf("NH3_dep_per_reserve.nc")

print("✓  Wrote NH3_dep_per_reserve.csv and NH3_dep_per_reserve.nc")

/var/folders/41/y5r5p6sj4_s8wprrtlzt0s_40000gp/T/ipykernel_76554/3025602590.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["area_km2"] = gdf.geometry.area * (111**2)        # rough km² in lat/lon


ValueError: conflicting sizes for dimension 'reserve': length 3 on the data but length 18 on coordinate 'reserve'

#!/usr/bin/env python
"""
Monthly NH₃ dry + wet deposition aggregated for *all 18* Natura-2000 bog reserves
"""

import geopandas as gpd
import xarray as xr
import numpy as np
import pandas as pd
import regionmask

# ------------------------------------------------------------------ #
# 1.  Geometry: read *all* 18 reserves                               #
# ------------------------------------------------------------------ #
shp = "spain_natura_2000b_bogs/spain_natura_2000b_bogs.shp"
gdf = gpd.read_file(shp).to_crs("EPSG:4326")                 # ensure lat/lon
gdf["id"] = range(1, len(gdf) + 1)
gdf["name"] = gdf.get("SITENAME", gdf["id"]).str.strip()

# ------------------------------------------------------------------ #
# 2.  NH₃ monthly deposition cube                                    #
# ------------------------------------------------------------------ #
ds_mo = xr.open_dataset("NH3_dry_wet_dep_2019-2022.nc")      # dims time,lat,lon
lon_dim, lat_dim = "lon", "lat"                             # confirm names

# ------------------------------------------------------------------ #
# 3.  Make a 2-D region mask                                        #
# ------------------------------------------------------------------ #
reg      = regionmask.from_geopandas(gdf, numbers="id", names="name", name="n2000")  
mask_2d  = reg.mask(ds_mo)                                   # new API infers lat/lon

# ------------------------------------------------------------------ #
# 4.  Grid-cell area (m²) – rectangular 0.1° grid                   #
# ------------------------------------------------------------------ #
R      = 6_371_000
dlat   = np.deg2rad(np.diff(ds_mo[lat_dim])[0])
dlon   = np.deg2rad(np.diff(ds_mo[lon_dim])[0])
latfac = np.cos(np.deg2rad(ds_mo[lat_dim]))
area   = (R**2 * dlat * dlon) * latfac.broadcast_like(ds_mo.NH3_dry_dep.isel(time=0))  

# ------------------------------------------------------------------ #
# 5.  Loop over months & reserves                                    #
# ------------------------------------------------------------------ #
records = []
for t in ds_mo.time:
    y, m = int(t.dt.year), int(t.dt.month)
    for rid, rname in zip(gdf.id, gdf.name):
        sel = (mask_2d == rid)
        dry_kg = (ds_mo.NH3_dry_dep.sel(time=t)*area).where(sel).sum().item()
        wet_kg = (ds_mo.NH3_wet_dep.sel(time=t)*area).where(sel).sum().item()
        records.append(dict(year=y, month=m, reserve=rname,
                            dry_kg=dry_kg, wet_kg=wet_kg,
                            total_kg=dry_kg + wet_kg))

df = pd.DataFrame(records)
df.to_csv("NH3_dep_per_reserve_monthly.csv", index=False)

# ------------------------------------------------------------------ #
# 6.  NetCDF (time × reserve)                                        #
# ------------------------------------------------------------------ #
# reserves = gdf.name.to_list()
# times    = ds_mo.time

# da_dry = xr.DataArray(np.nan, coords=[times, reserves],
#                       dims=["time", "reserve"], name="dry_dep_kg")
# da_wet = xr.DataArray(np.nan, coords=[times, reserves],
#                       dims=["time", "reserve"], name="wet_dep_kg")

# for rec in records:
#     ts = rec["timestamp"]            # store exact timestamp in records, OR:
#     # ts = rec["t"]                  # if you keep 't' directly
#     da_dry.loc[dict(time=ts, reserve=rec["reserve"])] = rec["dry_kg"]
#     da_wet.loc[dict(time=ts, reserve=rec["reserve"])] = rec["wet_kg"]

    
# xr.Dataset({"dry_dep_kg": da_dry, "wet_dep_kg": da_wet}) \
#   .to_netcdf("NH3_dep_per_reserve_monthly.nc")

# print("✓ Wrote CSV and NetCDF for 18 reserves × 41 months")

In [21]:
#!/usr/bin/env python
"""
Monthly NH₃ dry + wet deposition over 18 Natura-2000 bog reserves
*with* fractional cell overlap
"""

import geopandas as gpd
import xarray as xr
import numpy as np
import pandas as pd
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_origin
from tqdm import tqdm                           # progress bar (optional)

# ------------------------------------------------------------------ #
# 1.  Shapefile  (18 bogs)                                           #
# ------------------------------------------------------------------ #
shp = "spain_natura_2000b_bogs/spain_natura_2000b_bogs.shp"
gdf = gpd.read_file(shp).to_crs("EPSG:4326")
gdf["id"]   = range(1, len(gdf) + 1)
gdf["name"] = gdf.get("SITENAME", gdf["id"]).astype(str).str.strip()

# ------------------------------------------------------------------ #
# 2.  NH₃ monthly deposition cube                                    #
# ------------------------------------------------------------------ #
ds = xr.open_dataset("NH3_dry_wet_dep_2019-2022.nc")     # vars: NH3_dry_dep, NH3_wet_dep
lat, lon = ds.lat.values, ds.lon.values
dlat, dlon = np.diff(lat)[0], np.diff(lon)[0]

# GeoTransform for rasterio (upper-left corner, pixel size in degrees)
transform = from_origin(west=lon.min() - dlon/2,
                        north=lat.max() + dlat/2,
                        xsize=dlon, ysize=dlat)

height, width = len(lat), len(lon)

# ------------------------------------------------------------------ #
# 3.  Pre-compute cell area (m²) once                                 #
# ------------------------------------------------------------------ #
R = 6_371_000
cell_area = (
    (R**2 * np.deg2rad(dlat) * np.deg2rad(dlon))
    * np.cos(np.deg2rad(lat))[:, None]         # broadcast to 2-D
)

# ------------------------------------------------------------------ #
# 4.  Rasterise each reserve – boolean + fractional mask             #
# ------------------------------------------------------------------ #
supersample = 10                 # 10×10 sub-pixels → ≈1 km equivalent
fractions = {}                   # id → 2-D numpy array

for rid, geom in zip(gdf.id, gdf.geometry):
    # supersample raster, then downscale to fraction (0–1)
    high = rasterize(
        [(geom, 1)],
        out_shape=(height*supersample, width*supersample),
        transform=transform * transform.scale(1/supersample, 1/supersample),
        all_touched=False,       # stricter: only pixels whose centre in polygon
        dtype="uint8",
    )
    frac = high.reshape(height, supersample, width, supersample).mean(axis=(1,3))
    fractions[rid] = frac.astype(np.float32)    # store

# ------------------------------------------------------------------ #
# 5.  Loop over months & reserves                                    #
# ------------------------------------------------------------------ #
records = []

for t in tqdm(ds.time.values, desc="Months"):
    dry = ds.NH3_dry_dep.sel(time=t).values     # kg m-2
    wet = ds.NH3_wet_dep.sel(time=t).values

    for rid, name in zip(gdf.id, gdf.name):
        f = fractions[rid]                     # 2-D fraction (0–1)
        dry_kg = np.nansum(dry * cell_area * f)
        wet_kg = np.nansum(wet * cell_area * f)
        ts = np.datetime64(t)
        records.append(dict(
            time   = ts,
            year   = int(pd.to_datetime(ts).year),
            month  = int(pd.to_datetime(ts).month),
            reserve=name,
            dry_kg = dry_kg,
            wet_kg = wet_kg,
            total_kg=dry_kg + wet_kg,
        ))

# ------------------------------------------------------------------ #
# 6.  Save tidy CSV                                                  #
# ------------------------------------------------------------------ #
df = pd.DataFrame(records)
df.to_csv("NH3_dep_per_reserve_monthly.csv", index=False)

# ------------------------------------------------------------------ #
# 7.  Pivot → NetCDF (time × reserve)                                #
# ------------------------------------------------------------------ #
ds_out = (
    df.set_index(["time", "reserve"])
      .to_xarray()[["dry_kg", "wet_kg"]]
      .rename({"dry_kg": "dry_dep_kg", "wet_kg": "wet_dep_kg"})
)
ds_out.to_netcdf("NH3_dep_per_reserve_monthly.nc")

print("✓ wrote CSV & NetCDF with fractional-area weighting")

Months: 100%|██████████| 41/41 [00:00<00:00, 611.17it/s]

✓ wrote CSV & NetCDF with fractional-area weighting
